# Geometrical Methods in Machine Learning
## Seminar 3: Intrinsic Dimension Estimation

In [ ]:
from __future__ import print_function

%matplotlib inline
import matplotlib as mpl
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D

import numpy as np
from math import pi
from sklearn.datasets import load_digits, fetch_olivetti_faces
from sklearn.neighbors import NearestNeighbors

## 1. Maximum Likelihood

MLE is the average of estimators $\hat{m}_k$ for several values of $k$ where $\hat{m}_k$ in turn is the average of local maximum likelihood estimators $\hat{m}_k(x_i)$, which estimate the dimension of the data around a sample point $x_i$ based on the distances between $x_i$ and its $k$ nearest neighbor.

In [ ]:
"""
Implementation of 'Maximum Likelihood Estimation of Intrinsic Dimension' by Elizaveta Levina and Peter J. Bickel
 
how to use
----------
 
The goal is to estimate intrinsic dimensionality of data, the estimation of dimensionality is scale dependent
(depending on how much you zoom into the data distribution you can find different dimesionality), so they
propose to average it over different scales, the interval of the scales [k1, k2] are the only parameters of the algorithm.
 
This code also provides a way to repeat the estimation with bootstrapping to estimate uncertainty.
 
Here is one example with swiss roll :
 
from sklearn.datasets import make_swiss_roll
X, _ = make_swiss_roll(1000)
 
k1 = 10 # start of interval(included)
k2 = 20 # end of interval(included)
intdim_k_repeated = repeated(intrinsic_dim_scale_interval, 
                             X, 
                             mode='bootstrap', 
                             nb_iter=500, # nb_iter for bootstrapping
                             verbose=1, 
                             k1=k1, k2=k2)
intdim_k_repeated = np.array(intdim_k_repeated)
# the shape of intdim_k_repeated is (nb_iter, size_of_interval) where 
# nb_iter is number of bootstrap iterations (here 500) and size_of_interval
# is (k2 - k1 + 1).
 
# Plotting the histogram of intrinsic dimensionality estimations repeated over
# nb_iter experiments
plt.hist(intdim_k_repeated.mean(axis=1))
 
"""
# from tqdm import tqdm
import pandas as pd
import numpy as np
from sklearn.neighbors import NearestNeighbors
 
def intrinsic_dim_sample_wise(X, k=5):
    neighb = NearestNeighbors(n_neighbors=k+1).fit(X)
    dist, ind = neighb.kneighbors(X)
    dist = dist[:, 1:]
    dist = dist[:, 0:k]
    assert dist.shape == (X.shape[0], k)
    assert np.all(dist > 0)
    d = np.log(dist[:, k - 1: k] / dist[:, 0:k-1])
    d = d.sum(axis=1) / (k - 2)
    d = 1. / d
    intdim_sample = d
    return intdim_sample
 
def intrinsic_dim_scale_interval(X, k1=10, k2=20):
    X = pd.DataFrame(X).drop_duplicates().values # remove duplicates in case you use bootstrapping
    intdim_k = []
    for k in range(k1, k2 + 1):
        m = intrinsic_dim_sample_wise(X, k).mean()
        intdim_k.append(m)
    return intdim_k
 
def repeated(func, X, nb_iter=100, random_state=None, mode='bootstrap', **func_kw):
    if random_state is None:
        rng = np.random
    else:
        rng = np.random.RandomState(random_state)
    nb_examples = X.shape[0]
    results = []
 
    iters = range(nb_iter) 
    for i in iters:
        if mode == 'bootstrap':
            Xr = X[rng.randint(0, nb_examples, size=nb_examples)]
        elif mode == 'shuffle':
            ind = np.arange(nb_examples)
            rng.shuffle(ind)
            Xr = X[ind]
        elif mode == 'same':
            Xr = X
        else:
            raise ValueError('unknown mode : {}'.format(mode))
        results.append(func(Xr, **func_kw))
    return results

### 1.1 Artificial data

(Noisy) helix curve is given by parametric equation:

$$H(t) = \big( \sin(2 \pi t), \cos(2\pi t), t\big) + \varepsilon \\
\varepsilon \sim \mathcal{N}(0, \sigma)
$$

In [ ]:
def helix(t, var=0.02):
    # your code here
    x = 
    y = 
    z = 
    return 

In [ ]:
n = 500 # n samples
linspace = np.linspace(-1, 1, n)

data_helix = helix(linspace)
data_helix.shape

In [ ]:
fig = plt.figure(figsize=(6.5,6))
ax = fig.gca(projection='3d')
ax.scatter(data_helix[0], data_helix[1], data_helix[2], alpha=0.95, c=((data_helix[2]+1)*255), cmap="Reds")
plt.show()

#### Exercise

Estimate sample dimensionality using repeated function and given parameters. Plot histogram, print mean values and std. It could be comfortable to make histogram in a new cell. Useful functions: repeated, matplotlib.pyplot.hist=plt.hist 

In [ ]:
k1 = 3 # start of interval(included)
k2 = 50 # end of interval(included)
nb_iter = 10

sample = data_helix

intdim_k_repeated = repeated(intrinsic_dim_scale_interval, 
                             data_helix.T, 
                             mode='bootstrap', 
                             nb_iter=nb_iter, # nb_iter for bootstrapping 
                             k1=k1, k2=k2)
intdim_k_repeated = np.array(intdim_k_repeated)

In [ ]:
fig = plt.figure(figsize=(12,5.25))

plt.suptitle("Intrinsic dimension estimate via MLE")

plt.subplot(121)
plt.xlabel("Neighborhood cardinality")
plt.ylabel("Intrinsic dimension")
plt.grid(linestyle='dotted')

plt.plot(range(k1, k2 + 1), np.mean(intdim_k_repeated, axis=0), 'b')
plt.plot(range(k1, k2 + 1), np.mean(intdim_k_repeated, axis=0) + np.std(intdim_k_repeated, axis=0), 'r')
plt.plot(range(k1, k2 + 1), np.mean(intdim_k_repeated, axis=0) - np.std(intdim_k_repeated, axis=0), 'r')

plt.subplot(122)
plt.xlabel("Intrinsic dimension")
plt.grid(linestyle="dotted")

plt.hist(intdim_k_repeated.mean(axis=0))
plt.show()

#### Exercise

1. Plot mean value and mean value +- std as a function of a sample size.
2. Plot mean value and mean value +- std as a function of a log_10(noise_std).

In [ ]:
# your code here

### 1.2. Airfoils

#### Exercise

Estimate intrinsic dimension of airfoils sample and compare results with PCA ones.

In [ ]:
data_airfoils = np.loadtxt('../seminar1/data/airfoils.csv', delimiter=',')
data_airfoils.shape

In [ ]:
k1 = 3 # start of interval(included)
k2 = 50 # end of interval(included)
nb_iter = 20

intdim_k_repeated = repeated(intrinsic_dim_scale_interval, 
                             data_airfoils, 
                             mode='bootstrap', 
                             nb_iter=nb_iter, # nb_iter for bootstrapping 
                             k1=k1, k2=k2)
intdim_k_repeated = np.array(intdim_k_repeated)

In [ ]:
fig = plt.figure(figsize=(12,5.25))

plt.suptitle("Intrinsic dimension estimate via MLE")

plt.subplot(121)
plt.xlabel("Neighborhood cardinality")
plt.ylabel("Intrinsic dimension")
plt.grid(linestyle='dotted')

plt.plot(range(k1, k2 + 1), np.mean(intdim_k_repeated, axis=0), 'b')
plt.plot(range(k1, k2 + 1), np.mean(intdim_k_repeated, axis=0) + np.std(intdim_k_repeated, axis=0), 'r')
plt.plot(range(k1, k2 + 1), np.mean(intdim_k_repeated, axis=0) - np.std(intdim_k_repeated, axis=0), 'r')

plt.subplot(122)
plt.xlabel("Intrinsic dimension")
plt.grid(linestyle="dotted")

plt.hist(intdim_k_repeated.mean(axis=0))
plt.show()

### 1.3 MNIST

#### Exercise

Estimate intrinsic dimension of digits sample and compare results with PCA ones

In [ ]:
data_mnist = load_digits().data
data_mnist.shape

In [ ]:
k1 = 3 # start of interval(included)
k2 = 50 # end of interval(included)
nb_iter = 5

intdim_k_repeated = repeated(intrinsic_dim_scale_interval, 
                             data_mnist, 
                             mode='bootstrap', 
                             nb_iter=nb_iter, # nb_iter for bootstrapping 
                             k1=k1, k2=k2)
intdim_k_repeated = np.array(intdim_k_repeated)

In [ ]:
fig = plt.figure(figsize=(12,5.25))

plt.suptitle("Intrinsic dimension estimate via MLE")

plt.subplot(121)
plt.xlabel("Neighborhood cardinality")
plt.ylabel("Intrinsic dimension")
plt.grid(linestyle='dotted')

plt.plot(range(k1, k2 + 1), np.mean(intdim_k_repeated, axis=0), 'b')
plt.plot(range(k1, k2 + 1), np.mean(intdim_k_repeated, axis=0) + np.std(intdim_k_repeated, axis=0), 'r')
plt.plot(range(k1, k2 + 1), np.mean(intdim_k_repeated, axis=0) - np.std(intdim_k_repeated, axis=0), 'r')

plt.subplot(122)
plt.xlabel("Intrinsic dimension")
plt.grid(linestyle="dotted")

plt.hist(intdim_k_repeated.mean(axis=0))
plt.show()

### 1.4 Olivetti faces

#### Exercise

Estimate intrinsic dimension of faces sample and compare results with PCA ones.

In [ ]:
data_faces = fetch_olivetti_faces(shuffle=True).data
data_faces.shape

In [ ]:
k1 = 3 # start of interval(included)
k2 = 50 # end of interval(included)
nb_iter = 5

intdim_k_repeated = repeated(intrinsic_dim_scale_interval, 
                             data_faces, 
                             mode='bootstrap', 
                             nb_iter=nb_iter, # nb_iter for bootstrapping 
                             k1=k1, k2=k2)
intdim_k_repeated = np.array(intdim_k_repeated)

In [ ]:
fig = plt.figure(figsize=(12,5.25))

plt.suptitle("Intrinsic dimension estimate via MLE")

plt.subplot(121)
plt.xlabel("Neighborhood cardinality")
plt.ylabel("Intrinsic dimension")
plt.grid(linestyle='dotted')

plt.plot(range(k1, k2 + 1), np.mean(intdim_k_repeated, axis=0), 'b')
plt.plot(range(k1, k2 + 1), np.mean(intdim_k_repeated, axis=0) + np.std(intdim_k_repeated, axis=0), 'r')
plt.plot(range(k1, k2 + 1), np.mean(intdim_k_repeated, axis=0) - np.std(intdim_k_repeated, axis=0), 'r')

plt.subplot(122)
plt.xlabel("Intrinsic dimension")
plt.grid(linestyle="dotted")

plt.hist(intdim_k_repeated.mean(axis=0))
plt.show()

## 2. Local PCA

Let $\{x_1, x_2, \dots, x_n \} \subset \mathbb{R}^D$ be a data sample and $U(x_i)$ - a neighborhood of each point, excluding $x_i$ itself, with cardinality $k$. By applying PCA locally, i.e. computing eigenvalues $\lambda_{i,1} \geq \lambda_{i,2} \geq \dots \lambda_{i,D}$ of each neighborhood covariance matrix: $\mathbf{S}_i = \frac{1}{k-1}(\mathbf{U}(x_i) - x_i) (\mathbf{U}(x_i) - x_i)^T$ and then averaging over eigenvalues $\lambda_j = n^{-1} \sum_{i=1}^n \lambda_{i,j}$, where $j = 1, 2, \dots, D$ one can estimate intrinsic dimension $d$ by thresholding variance explained with averaged eigenvalues.

#### Exercise

1. Implement Local PCA method for intrinsic dimensionality estimation (lecture slides, pp. 28-32).  
2. Apply the method to artificial and real data, compare results with global PCA and MLE methods for different number of nearest neighbors $k$, conclude.

For neighborhood selection you may use `kneigbbors` method from scikit-learn's `NearestNeighbors`.

In [ ]:
# k of nearest neighbors to select
k = 9

# your code here

#### Exercise

Compute explained and cumulative explained variance. Use results to estimate intrinsic dimension of data samples.

$$EV_i := \frac{\lambda_i}{\lambda_1 + \dots + \lambda_D}$$
$$CEV_i := \frac{\sum_{i=1}^{d} \lambda_i}{\sum_{j=1}^{D} \lambda_j}$$

In [ ]:
EV = 
CEV = 

In [ ]:
# plot EV/CEVs
fig = plt.figure(figsize=(12,5.25))

plt.subplot(121)
plt.title("Explained variance")
plt.xlabel("# PCs")
plt.grid(linestyle="dotted")
plt.plot(EV, "o-")

plt.subplot(122)
plt.title("Cumulative explained variance")
plt.axhline(linewidth=1, y=0.99, color='r')
plt.axhline(linewidth=1, y=0.95, color='r')
plt.axhline(linewidth=1, y=0.9, color='r')
plt.axhline(linewidth=1, y=0.8, color='r')
plt.xlabel("# PCs")
plt.grid(linestyle="dotted")
plt.plot(CEV, "o-")